This is a little mnist fashion classifier, via [this lesson](https://www.youtube.com/watch?v=GVKDa5hxUZE).

In [18]:
# import keras packages
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [2]:
# logging code
import wandb
from wandb.keras import WandbCallback
run = wandb.init(project="fashion")

wandb: ERROR To use wandb on Windows, you need to run the command "wandb run python <your_train_script>.py"


In [3]:
# load data
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
img_width  = X_train.shape[1]
img_height = X_train.shape[2]

In [4]:
# one hot encode the outputs
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

labels = ["T-shirt/top", "Trouser", "Pullover", "Dress",
          "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

num_classes = y_train.shape[1]

In [5]:
# create and fit the model
model = Sequential()
model.add(Flatten(input_shape=(img_width,img_height)))
model.add(Dense(num_classes))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test),
          callbacks=[WandbCallback(data_type="image", labels=labels)])

W0925 16:51:05.160989 10236 deprecation.py:506] From C:\Users\jfields\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
wandb: ERROR wandb.init hasn't been called, can't configure run
W0925 16:51:05.317957 10236 deprecation.py:323] From C:\Users\jfields\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 50us/sample - loss: 11.3480 - acc: 0.3345 - val_loss: 11.0490 - val_acc: 0.3666
Epoch 2/10
60000/60000 [==============================] - 3s 48us/sample - loss: 11.0955 - acc: 0.3720 - val_loss: 10.6912 - val_acc: 0.3748
Epoch 3/10
60000/60000 [==============================] - 3s 48us/sample - loss: 9.8708 - acc: 0.3587 - val_loss: 10.5332 - val_acc: 0.3295
Epoch 4/10
60000/60000 [==============================] - 2s 42us/sample - loss: 9.7972 - acc: 0.2815 - val_loss: 8.6312 - val_acc: 0.2864
Epoch 5/10
60000/60000 [==============================] - 2s 38us/sample - loss: 8.6753 - acc: 0.2914 - val_loss: 8.6312 - val_acc: 0.2864
Epoch 6/10
60000/60000 [==============================] - 3s 45us/sample - loss: 8.6753 - acc: 0.2914 - val_loss: 8.6312 - val_acc: 0.2864
Epoch 7/10
60000/60000 [==============================] - 2s 38us/sample - loss: 8.6753 - acc: 0.2914 - val_los

29% accuracy.....kinda sucks!

There are some improvements we can make though. 

First off, lets normalize the data (after reimporting it)

In [16]:
# load data
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
img_width = X_train.shape[1]
img_height = X_train.shape[2]

# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
labels = ["T-shirt/top", "Trouser", "Pullover", "Dress",
          "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]


X_train = X_train / 255.
X_test = X_test / 255.

num_classes = y_train.shape[1]

# create model
model = Sequential()
model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dense(num_classes, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test),
          callbacks=[WandbCallback(data_type="image", labels=labels)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.6017 - acc: 0.7962 - val_loss: 0.5051 - val_acc: 0.8272
Epoch 2/10
60000/60000 [==============================] - 3s 43us/sample - loss: 0.4615 - acc: 0.8420 - val_loss: 0.4731 - val_acc: 0.8358
Epoch 3/10
60000/60000 [==============================] - 4s 68us/sample - loss: 0.4356 - acc: 0.8501 - val_loss: 0.4590 - val_acc: 0.8387
Epoch 4/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.4210 - acc: 0.8553 - val_loss: 0.4581 - val_acc: 0.8396
Epoch 5/10
60000/60000 [==============================] - 3s 53us/sample - loss: 0.4139 - acc: 0.8565 - val_loss: 0.4589 - val_acc: 0.8399
Epoch 6/10
60000/60000 [==============================] - 3s 53us/sample - loss: 0.4065 - acc: 0.8595 - val_loss: 0.4515 - val_acc: 0.8441
Epoch 7/10
60000/60000 [==============================] - 3s 52us/sample - loss: 0.4014 - acc: 0.8610 - val_loss: 0.

86% on the data! Sweet. Normalizing images is a must.

Note - our training accuracy ends higher than our validation accuracy. This is a telltale sign of overfitting.

Next, we're going to add another activation function (relu) and a few dropout layers - the latter helping with overfitting.

In [19]:
# create model
model = Sequential()
model.add(Flatten(input_shape=(img_width, img_height)))
model.add(Dropout(0.4))
model.add(Dense(100, activation="relu")) # Added - 100=num hidden nodes
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test),
          callbacks=[WandbCallback(data_type="image", labels=labels)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.6953 - acc: 0.7485 - val_loss: 0.4781 - val_acc: 0.8291
Epoch 2/10
60000/60000 [==============================] - 6s 101us/sample - loss: 0.5353 - acc: 0.8038 - val_loss: 0.4374 - val_acc: 0.8455
Epoch 3/10
60000/60000 [==============================] - 7s 116us/sample - loss: 0.5066 - acc: 0.8145 - val_loss: 0.4303 - val_acc: 0.8398
Epoch 4/10
60000/60000 [==============================] - 6s 94us/sample - loss: 0.4878 - acc: 0.8203 - val_loss: 0.4111 - val_acc: 0.8501
Epoch 5/10
60000/60000 [==============================] - 5s 90us/sample - loss: 0.4766 - acc: 0.8248 - val_loss: 0.4045 - val_acc: 0.8507
Epoch 6/10
60000/60000 [==============================] - 7s 123us/sample - loss: 0.4712 - acc: 0.8270 - val_loss: 0.4001 - val_acc: 0.8514
Epoch 7/10
60000/60000 [==============================] - 6s 97us/sample - loss: 0.4623 - acc: 0.8299 - val_loss

Poorer training accuracy but an increased validation accuracy. Nice.